In [1]:
from obspy import UTCDateTime
from obspy import read_inventory
from obspy.clients.filesystem.sds import Client as wClient
from obspy.clients.fdsn import Client as nClient
import numpy as np
import math
from matplotlib import mlab
from matplotlib.dates import date2num
from scipy import signal
import scipy.io
from obspy.clients.fdsn.header import FDSNNoDataException

In [2]:
# projection for ET sensitivity
# A = PSD displacement
def ET_proj(F,A):
    rho0=2.7e3
    G=6.67e-11
    armL=10000
    h_NN = (1/((2*np.pi*F)**4))*np.sqrt(A)*(4*np.pi/3)*rho0*G*2*np.sqrt(2)/(armL)
    return h_NN

In [3]:
# Start and Stop date/time
startdate=UTCDateTime('2021-12-21T00:00:00.000')
stopdate= UTCDateTime('2023-12-21T00:00:00.000') # P2...
#startdate=UTCDateTime('2023-09-13T00:00:00.000') # Terziet
#stopdate= UTCDateTime('2023-08-14T00:00:00.000')
#stopdate= UTCDateTime('2023-11-20T00:00:00.000')

# Name of data channel
#network='ET'
network='NL' # Terziet
## 00 is the surface seismometer, 01 is the uderground
location='01'
#location='00'
#location=''
#station='P2'
#station='P3'
#station='SOE0'
#station='SOE2'
station='TERZ' # Terziet
#channel='HH1'
#channel='HH2'
#channel='HHN'
#channel='HHE'
channel='HHZ'
FullName=network+'.'+station+'.'+location+'.'+channel

#remote=0
remote=1 # Terziet

# Set savespec to 1 to save the spectra also
savespec=0

In [4]:
#Data Path
Path='/mnt/sites-data/sosenattos/ETSCOPE'
dist='KNMI'
if remote==0:
    waves=wClient(Path)
    invfile='ETScope.xml'    
else:
    waves=nClient(dist)
    invfile='TERZ_network.xml'

In [5]:
# Sampling frequency
st=waves.get_waveforms(station=station,channel=channel,network=network,location=location,starttime=startdate,endtime=startdate+1)
st.merge()
tr=st[0]
fs=tr.stats.sampling_rate
print(tr.stats.sampling_rate)
delta=1.0/fs

200.0


In [6]:
# Inventory

inv=read_inventory(invfile)

In [7]:
# Number of day of each block
Nday = 1
# Time duration of each block of data extraced in the archive
TLong=24*3600*Nday
# Interval for saving spectra
Tsave=24*3600*Nday
# Time duration of data used to compute a power spectrum (eventually it is an average of severl periodograms)
T=120
# Overlap of each power spectrum
Ovl=0.5
# Time duration of a single periodogram
Tfft=120
# Overlap for each periodogram
Ovlfft=0.5
# minimum and maximum frequency to integrate
fmin=0.1
fmax=50
NN_fmin=2
NN_fmin1=3
NN_fmax=10

In [8]:
dT=TLong+T*Ovl

Num=int(T*fs)
Numfft=int(Tfft*fs)
NOvl=int(Ovlfft*Numfft)

M=int((dT-T)/(T*(1-Ovl))+1)

K=int((stopdate-startdate)/(T*(1-Ovl)))
Ksave=int((Tsave)/(T*(1-Ovl)))

In [9]:
_,f=mlab.psd(np.ones(Numfft),NFFT=Numfft,Fs=fs)

df=f[1]

# find index for minimum and maximum frequencies to save spectra
imin = (np.abs(f-fmin)).argmin()
imax = (np.abs(f-fmax)).argmin()
# and to compute the projection of NN on ET
NNimin = (np.abs(f-NN_fmin)).argmin()
NNimin1 = (np.abs(f-NN_fmin1)).argmin()
NNimax = (np.abs(f-NN_fmax)).argmin()

t=startdate

v=np.zeros((Ksave,imax-imin+1))
v_NN_Ros=np.zeros((Ksave,imax-imin+1))
v_NN1_Ros=np.zeros((Ksave,imax-imin+1))
v_NN=np.zeros((Ksave,imax-imin+1))
v_NN1=np.zeros((Ksave,imax-imin+1))

ET_NN=np.empty(K)
ET_NN1=np.empty(K)

ET_DR=np.empty(K)
ET_DR1=np.empty(K)

d=np.empty(K)
dsave=np.empty(Ksave)

k=0
ksave=0
Nsave=0

In [10]:
# load ET nominal sensitivity
ET_sens=scipy.io.loadmat('ET_D.mat')
ET_f=ET_sens['freq'][0]
ET_HF=ET_sens['ET_D_HF_strain_ASD'][0]
ET_LF=ET_sens['ET_D_LF_strain_ASD'][0]
ET_ASD_1=np.minimum(ET_LF,ET_HF)
ET_ASD=np.interp(f,ET_f,ET_ASD_1)

In [11]:
# load ET computed with and without NN
ET_Noise_file = scipy.io.loadmat('ETNoiseNN.mat')
ET_Noise_f = ET_Noise_file['Freq_ETall'][0]
ASD_ETall = ET_Noise_file['ASD_ETall'][0]
PSD_ETLF_noNN = ET_Noise_file['PSD_ETLF_noNN'][0]
PSD_ETHF_noNN = ET_Noise_file['PSD_ETHF_noNN'][0]
ASD_ET_int = np.interp(f,ET_Noise_f,ASD_ETall)
PSD_ETLF_noNN_int = np.interp(f,ET_Noise_f,PSD_ETLF_noNN)
PSD_ETHF_noNN_int = np.interp(f,ET_Noise_f,PSD_ETHF_noNN)

In [12]:
badday = 0

while t<stopdate:
    while True:
        try:
            st=waves.get_waveforms(station=station,channel=channel,network=network,location=location,starttime=t,endtime=t+dT-delta)
            st.merge()
            break
        except FDSNNoDataException:
            print("No data available for the specified parameters.") #urllib.error.URLError: # URLError:
            #print('Problems in connection: retry...')
            break
            #URLError:
            #print('Problems in connection: retry...')
    print('Processing from date: ',t)
    t1=t
    response=inv.get_response(FullName,t1)
    response_old=response
    resp,_=response.get_evalresp_response(t_samp=delta,nfft=Numfft,output='VEL')
    TF=abs(resp)**2
    TF[0]=1e-16
    for n in range(0,M):
        if ksave==0:
            tsave=t1
        v[ksave]=np.nan
        ET_NN[k]=np.nan
        ET_NN1[k]=np.nan
        t2=t1+T-delta
        tr=st.slice(t1,t2)
        if tr.get_gaps()==[] and len(tr)>0:
            tr1=tr[0]
            #print(tr1.stats.npts,Numfft)
            if tr1.stats.npts==Num:
                response=inv.get_response(FullName,t1)
                if response!=response_old:
                    print('Calibration change detected at: ', t1)
                    response_old=response
                    # get a response in VEL
                    resp,_=response.get_evalresp_response(t_samp=delta,nfft=Numfft,output='VEL')
                    TF=abs(resp)**2
                    TF[0]=1e-16
                # PSD
                s,_=mlab.psd(tr1.data,NFFT=Numfft,noverlap=NOvl,Fs=fs,detrend="linear")
                # move from PSD to PSD in VEL and then in displacement (2*pi*f)
                s=(2*np.pi*f)**2*s/TF
                # PSD displacement
                v[ksave]=s[imin:imax+1]
                
                # fmin = 2 Hz Rosario code
                proj=ET_proj(f[NNimin:NNimax+1],s[NNimin:NNimax+1])
                ET_NN[k]=np.sqrt(1.0/(NN_fmax-NN_fmin)*df*np.sum((proj/ET_ASD[NNimin:NNimax+1])**2))
                # fmin = 3 Hz
                proj1=ET_proj(f[NNimin1:NNimax+1],s[NNimin1:NNimax+1])
                ET_NN1[k]=np.sqrt(1.0/(NN_fmax-NN_fmin1)*df*np.sum((proj1/ET_ASD[NNimin1:NNimax+1])**2))
                
                # fmin = 2 Hz Davide modification
                PSD_ETLF = proj**2 + PSD_ETLF_noNN_int[NNimin:NNimax+1]
                PSD_ETHF = proj**2 + PSD_ETHF_noNN_int[NNimin:NNimax+1]
                PSD_ETLHF = 1./(1./PSD_ETLF + 1./PSD_ETHF)
                ET_DR[k]=np.sqrt(1.0/(NN_fmax-NN_fmin)*df*np.sum(PSD_ETLHF/(ASD_ET_int[NNimin:NNimax+1])**2))
                # fmin = 3 Hz
                PSD_ETLF1 = proj1**2 + PSD_ETLF_noNN_int[NNimin1:NNimax+1]
                PSD_ETHF1 = proj1**2 + PSD_ETHF_noNN_int[NNimin1:NNimax+1]
                PSD_ETLHF1 = 1./(1./PSD_ETLF1 + 1./PSD_ETHF1)
                ET_DR1[k]=np.sqrt(1.0/(NN_fmax-NN_fmin1)*df*np.sum(PSD_ETLHF1/(ASD_ET_int[NNimin1:NNimax+1])**2))

        else :
            badday = badday+1
            #print(badday,t1,t2)
        
        d[k]=date2num(t1.datetime)
        dsave[ksave]=d[k]
        t1=t1+T*(1-Ovl)
        k=k+1
        ksave=ksave+1
        if ksave==Ksave:
            ksave=0
            if savespec==1:
                NameFile='Spectra_'+station+'/Spec_'+FullName+'.'+str(tsave.year)+'.'+str(tsave.month)+'.'+str(tsave.day).zfill(3)+'.mat'
                scipy.io.savemat(NameFile,mdict={'f':f[imin:imax+1],'d':dsave,'v':v})
    t=t+TLong

print(badday)

Processing from date:  2021-12-21T00:00:00.000000Z


 WARNING (norm_resp): computed and reported sensitivities differ by more than 5 percent. 
	 Execution continuing.


Processing from date:  2021-12-22T00:00:00.000000Z


 WARNING (norm_resp): computed and reported sensitivities differ by more than 5 percent. 
	 Execution continuing.


Processing from date:  2021-12-23T00:00:00.000000Z


...

No data available for the specified parameters.
Processing from date:  2023-12-19T00:00:00.000000Z


 WARNING (norm_resp): computed and reported sensitivities differ by more than 5 percent. 
	 Execution continuing.


No data available for the specified parameters.
Processing from date:  2023-12-20T00:00:00.000000Z


 WARNING (norm_resp): computed and reported sensitivities differ by more than 5 percent. 
	 Execution continuing.


85318


In [13]:
nday=int((stopdate-startdate)/86400)
NameFile='ET_DR_NN_Proj_'+FullName+'.'+str(startdate.year)+'.'+str(startdate.julday).zfill(3)+'_'+str(nday).zfill(4)+'.mat'
scipy.io.savemat(NameFile,mdict={'d':d, 'ET_NN':ET_NN, 'ET_NN1':ET_NN1, 'ET_DR':ET_DR, 'ET_DR1':ET_DR1})